# Style Transfer for robot movements

Uses robot movements (from CSV files) and human movement videos (ballet, jazz, etc.) 

Learns human movement styles by analysing both raw video and pose using mediapipe

Transfers the style of human movement to robot motions while keeping them similar

Few-shot adaptation is used to adapt to new styles with a few example videos

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import cv2
import os
import pickle
import glob
from sklearn.preprocessing import MinMaxScaler
import mediapipe as mp
from tqdm import tqdm
from scipy.stats import pearsonr
import matplotlib.pyplot as plt

video_folder = "robot_motion_data/human_videos" #Folder containing human motion videos
robot_folder = "robot_motion_data/robot_motions2/" #Folder containing robot 7DOF joint CSV files

## Data preprocessing

In [ ]:

def load_robot_csv(csv_path):
    df = pd.read_csv(csv_path)
    joint_columns = ['Position1', 'Position2', 'Position3', 'Position4', 
                    'Position5', 'Position6', 'Position7']
    joint_data = df[joint_columns].values.astype(np.float32)
    joint_data = joint_data[~np.isnan(joint_data).any(axis=1)]
    return joint_data

def create_robot_sequences(joint_data, sequence_length=128):
    sequences = []
    for i in range(0, len(joint_data) - sequence_length + 1, sequence_length // 2):
        sequence = joint_data[i:i + sequence_length]
        sequences.append(sequence)
    return np.array(sequences, dtype=np.float32)

def process_robot_folder(folder_path, sequence_length=128):
    csv_files = glob.glob(os.path.join(folder_path, "*.csv"))
    all_sequences = []
    all_raw_data = []
    
    for csv_file in csv_files:
        joint_data = load_robot_csv(csv_file)
        all_raw_data.append(joint_data)
        sequences = create_robot_sequences(joint_data, sequence_length)
        all_sequences.extend(sequences)
    
    # Fit scaler on all data
    combined_data = np.vstack(all_raw_data).astype(np.float32)
    scaler = MinMaxScaler(feature_range=(-1, 1))
    scaler.fit(combined_data)
    
    # Scale sequences
    scaled_sequences = []
    for seq in all_sequences:
        scaled_seq = scaler.transform(seq.astype(np.float32))
        scaled_sequences.append(scaled_seq.astype(np.float32))
    
    sequences = np.array(scaled_sequences, dtype=np.float32)
    return sequences, scaler

def load_video(video_path, target_size=(64, 64), sequence_length=128):
    cap = cv2.VideoCapture(video_path)
    frames = []
    
    while len(frames) < sequence_length:
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, target_size)
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame = frame.astype(np.float32) / 255.0
        frames.append(frame)
    
    cap.release()
    
    # Pad if necessary
    while len(frames) < sequence_length:
        frames.append(frames[-1])
    
    return np.array(frames[:sequence_length])

def extract_pose_sequence(video_frames):
    mp_pose = mp.solutions.pose.Pose(
        model_complexity=2,
        enable_segmentation=False,
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5
    )
    
    pose_sequences = []
    for frame in video_frames:
        frame_uint8 = (frame * 255).astype(np.uint8)
        results = mp_pose.process(frame_uint8)
        
        if results.pose_landmarks:
            keypoints = []
            for landmark in results.pose_landmarks.landmark:
                keypoints.extend([landmark.x, landmark.y, landmark.z, landmark.visibility])
            pose_sequences.append(keypoints)
        else:
            pose_sequences.append([0.0] * 132)  # 33 landmarks × 4 coordinates
    
    mp_pose.close()
    return np.array(pose_sequences, dtype=np.float32)

def process_video_folder_with_poses(video_folder, pose_folder=None):
    if pose_folder is None:
        pose_folder = os.path.join(video_folder, "..", "preprocessed_poses")
    
    os.makedirs(pose_folder, exist_ok=True)
    video_pose_mapping = {}
    
    for style_name in os.listdir(video_folder):
        style_path = os.path.join(video_folder, style_name)
        if not os.path.isdir(style_path):
            continue
        
        style_pose_folder = os.path.join(pose_folder, style_name)
        os.makedirs(style_pose_folder, exist_ok=True)
        
        video_files = []
        for ext in ['*.mp4', '*.avi', '*.mov', '*.mkv']:
            video_files.extend(glob.glob(os.path.join(style_path, ext)))
        
        if not video_files:
            continue
        
        video_pose_pairs = []
        for video_path in tqdm(video_files, desc=f"Processing {style_name}"):
            try:
                video_name = os.path.splitext(os.path.basename(video_path))[0]
                pose_file_path = os.path.join(style_pose_folder, f"{video_name}_poses.npy")
                
                if os.path.exists(pose_file_path):
                    video_pose_pairs.append((video_path, pose_file_path))
                    continue
                
                video_frames = load_video(video_path)
                poses = extract_pose_sequence(video_frames)
                np.save(pose_file_path, poses.astype(np.float32))
                video_pose_pairs.append((video_path, pose_file_path))
                
            except Exception as e:
                print(f"Failed to process {os.path.basename(video_path)}: {e}")
                continue
        
        if video_pose_pairs:
            video_pose_mapping[style_name] = video_pose_pairs
    
    # Save mapping
    mapping_path = os.path.join(pose_folder, "video_pose_mapping.pkl")
    with open(mapping_path, 'wb') as f:
        pickle.dump(video_pose_mapping, f)
    
    return video_pose_mapping

def process_robot_data():
    robot_sequences, scaler = process_robot_folder(robot_folder)
    
    np.save("processed_robot_sequences.npy", robot_sequences)
    with open("robot_scaler.pkl", 'wb') as f:
        pickle.dump(scaler, f)
    
    print(f"Processed {robot_sequences.shape[0]} robot sequences")
    return robot_sequences, scaler

def preprocess_videos():
    mapping = process_video_folder_with_poses(video_folder)
    print("Video preprocessing complete!")
    return mapping

In [ ]:
process_robot_data()
preprocess_videos()

## Model architecture - cross modal attention

In [ ]:

def extract_temporal_motion_features(video_input):
    
    # Calculate frame differences (motion) using Lambda layer
    def compute_motion_sequence(video):
        motion_frames = []
        for t in range(1, 128):
            prev_frame = video[:, t-1]
            curr_frame = video[:, t]
            motion = curr_frame - prev_frame
            motion_frames.append(motion)
        
        # Pad first frame
        motion_frames.insert(0, motion_frames[0])
        return tf.stack(motion_frames, axis=1)
    
    motion_sequence = tf.keras.layers.Lambda(compute_motion_sequence)(video_input)
    
    # Extract motion features per timestep
    motion_features = tf.keras.layers.TimeDistributed(
        tf.keras.layers.Conv2D(32, (8, 8), strides=(8, 8), activation='relu')
    )(motion_sequence)
    
    motion_features = tf.keras.layers.TimeDistributed(
        tf.keras.layers.GlobalAveragePooling2D()
    )(motion_features)
    
    return motion_features

def extract_temporal_pose_features(pose_input):
    
    # Reshape poses to (batch, time, joints, coordinates)
    pose_reshaped = tf.keras.layers.Reshape((128, 33, 4))(pose_input)
    pose_coords = tf.keras.layers.Lambda(lambda x: x[:, :, :, :3])(pose_reshaped)  # Only x, y, z coordinates
    
    # Calculate pose velocity and acceleration using Lambda layers
    def compute_pose_dynamics(pose_coords):
        # Calculate pose velocity (rhythm)
        pose_velocity = pose_coords[:, 1:] - pose_coords[:, :-1]
        pose_velocity = tf.pad(pose_velocity, [[0, 0], [0, 1], [0, 0], [0, 0]], mode='CONSTANT')
        
        # Calculate pose acceleration (dynamics)
        pose_acceleration = pose_velocity[:, 1:] - pose_velocity[:, :-1]
        pose_acceleration = tf.pad(pose_acceleration, [[0, 0], [0, 1], [0, 0], [0, 0]], mode='CONSTANT')
        
        return pose_velocity, pose_acceleration
    
    pose_velocity, pose_acceleration = tf.keras.layers.Lambda(compute_pose_dynamics)(pose_coords)
    
    # Extract movement magnitude and direction per timestep
    velocity_magnitude = tf.keras.layers.Lambda(lambda x: tf.reduce_mean(tf.abs(x), axis=[2, 3]))(pose_velocity)
    acceleration_magnitude = tf.keras.layers.Lambda(lambda x: tf.reduce_mean(tf.abs(x), axis=[2, 3]))(pose_acceleration)
    
    # Extract joint coordination patterns
    def compute_coordination_features(pose_velocity):
        coordination_features = []
        for joint_idx in range(0, 33, 4):  # Sample every 4th joint
            joint_vel = pose_velocity[:, :, joint_idx, :]
            joint_coordination = tf.reduce_mean(joint_vel, axis=-1)  # [batch, time]
            coordination_features.append(joint_coordination)
        return tf.stack(coordination_features, axis=-1)  # [batch, time, 8]
    
    coordination_tensor = tf.keras.layers.Lambda(compute_coordination_features)(pose_velocity)
    
    # Combine rhythm features
    def combine_rhythm_features(inputs):
        velocity_mag, acceleration_mag = inputs
        return tf.stack([velocity_mag, acceleration_mag], axis=-1)
    
    rhythm_features = tf.keras.layers.Lambda(combine_rhythm_features)([velocity_magnitude, acceleration_magnitude])
    
    # Combine all temporal features
    temporal_features = tf.keras.layers.Concatenate(axis=-1)([rhythm_features, coordination_tensor])
    
    return temporal_features

class CrossModalAttentionLayer(tf.keras.layers.Layer):
    
    def __init__(self, d_model=128, num_heads=4, rate=0.1, **kwargs):
        super(CrossModalAttentionLayer, self).__init__(**kwargs)
        self.d_model = d_model
        self.num_heads = num_heads
        self.rate = rate
        
        # Cross-attention layers
        self.video_to_pose_attention = tf.keras.layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=d_model//num_heads, dropout=rate
        )
        self.pose_to_video_attention = tf.keras.layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=d_model//num_heads, dropout=rate
        )
        
        # Layer normalization
        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm3 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm4 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        
        # Projection layers to ensure consistent dimensions
        self.video_projection = tf.keras.layers.Dense(d_model)
        self.pose_projection = tf.keras.layers.Dense(d_model)
        
        # Dropout
        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dropout2 = tf.keras.layers.Dropout(rate)
    
    def call(self, inputs, training=False):
        video_features, pose_features = inputs
        
        # Project to consistent dimensions
        video_proj = self.video_projection(video_features)
        pose_proj = self.pose_projection(pose_features)
        
        # Video attending to pose (video queries, pose keys/values)
        video_attended = self.video_to_pose_attention(
            query=video_proj, key=pose_proj, value=pose_proj, training=training
        )
        video_attended = self.dropout1(video_attended, training=training)
        video_out = self.layernorm1(video_proj + video_attended)  # Residual connection
        
        # Pose attending to video (pose queries, video keys/values)
        pose_attended = self.pose_to_video_attention(
            query=pose_proj, key=video_proj, value=video_proj, training=training
        )
        pose_attended = self.dropout2(pose_attended, training=training)
        pose_out = self.layernorm2(pose_proj + pose_attended)  # Residual connection
        
        return video_out, pose_out
    
    def get_config(self):
        config = super().get_config()
        config.update({
            'd_model': self.d_model,
            'num_heads': self.num_heads,
            'rate': self.rate
        })
        return config
    
def build_temporal_style_encoder(embed_dim=256, use_cross_attention=True):
    
    video_input = tf.keras.Input(shape=(128, 64, 64, 3), name='video_frames')
    pose_input = tf.keras.Input(shape=(128, 132), name='pose_keypoints')
    
    # Extract temporal features
    video_temporal = extract_temporal_motion_features(video_input)
    pose_temporal = extract_temporal_pose_features(pose_input)
    
    print(f"Video temporal shape expected: [batch, 128, 32]")
    print(f"Pose temporal shape expected: [batch, 128, 10]")
    
    if use_cross_attention:
        # Apply cross-modal attention
        cross_attention = CrossModalAttentionLayer(d_model=128, num_heads=4, rate=0.1)
        video_attended, pose_attended = cross_attention([video_temporal, pose_temporal])
        
        # Additional processing after attention
        video_processed = tf.keras.layers.Conv1D(64, 3, padding='same', activation='relu')(video_attended)
        pose_processed = tf.keras.layers.Conv1D(64, 3, padding='same', activation='relu')(pose_attended)
        
        # Combine attended features
        combined_features = tf.keras.layers.Concatenate(axis=-1)([video_processed, pose_processed])
        
        # Apply attention to the combined features to focus on important temporal moments
        temporal_attention = tf.keras.layers.MultiHeadAttention(
            num_heads=8, key_dim=16, dropout=0.1
        )(combined_features, combined_features)
        
        # Residual connection and normalization
        attended_combined = tf.keras.layers.LayerNormalization()(
            combined_features + temporal_attention
        )
        
        # Final processing
        style_conv1 = tf.keras.layers.Conv1D(128, 5, padding='same', activation='relu')(attended_combined)
        style_conv2 = tf.keras.layers.Conv1D(128, 3, padding='same', activation='relu')(style_conv1)
        
    else:
        # Fallback to simple concatenation
        combined_temporal = tf.keras.layers.Concatenate(axis=-1)([video_temporal, pose_temporal])
        
        # Process with temporal convolutions
        style_conv1 = tf.keras.layers.Conv1D(128, 5, padding='same', activation='relu')(combined_temporal)
        style_conv2 = tf.keras.layers.Conv1D(128, 3, padding='same', activation='relu')(style_conv1)
    
    # Apply attention to focus on important temporal moments
    attention_weights = tf.keras.layers.Dense(1, activation='sigmoid')(style_conv2)
    style_attended = tf.keras.layers.Multiply()([style_conv2, attention_weights])
    
    # Final temporal style features
    style_features = tf.keras.layers.Dense(embed_dim, activation='relu')(style_attended)
    
    model_name = 'temporal_style_encoder_with_attention' if use_cross_attention else 'temporal_style_encoder'
    return tf.keras.Model([video_input, pose_input], style_features, name=model_name)

class TemporalTransformerBlock(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, feed_forward_dim, rate=0.1):
        super(TemporalTransformerBlock, self).__init__()
        self.d_model = d_model
        self.num_heads = num_heads
        self.feed_forward_dim = feed_forward_dim
        self.rate = rate
        
        self.mha = tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=d_model)
        self.ffn = tf.keras.Sequential([
            tf.keras.layers.Dense(feed_forward_dim, activation='relu'),
            tf.keras.layers.Dense(d_model)
        ])
        
        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        
        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dropout2 = tf.keras.layers.Dropout(rate)
    
    def call(self, x, training=False):
        attn_output = self.mha(x, x, training=training)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(x + attn_output)
        
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        out2 = self.layernorm2(out1 + ffn_output)
        
        return out2

def build_temporal_style_transfer_transformer(sequence_length=128, num_joints=7, style_dim=256):
    
    robot_sequence = tf.keras.Input(shape=(sequence_length, num_joints), name='robot_sequence')
    style_sequence = tf.keras.Input(shape=(sequence_length, style_dim), name='style_sequence')
    
    # Combine robot motion with temporal style
    combined_input = tf.keras.layers.Concatenate(axis=-1)([robot_sequence, style_sequence])
    
    # Project to model dimension
    d_model = 256
    x = tf.keras.layers.Dense(d_model)(combined_input)
    
    # Add positional encoding
    def add_positional_encoding(inputs):
        x_input = inputs
        positions = tf.range(sequence_length, dtype=tf.float32)
        pos_encoding = tf.keras.layers.Embedding(sequence_length, d_model)(positions)
        return x_input + pos_encoding
    
    x = tf.keras.layers.Lambda(add_positional_encoding)(x)
    
    # Multi-layer transformer with moderate dropout
    x = TemporalTransformerBlock(d_model=d_model, num_heads=8, feed_forward_dim=512, rate=0.25)(x)
    x = TemporalTransformerBlock(d_model=d_model, num_heads=8, feed_forward_dim=512, rate=0.25)(x)
    x = TemporalTransformerBlock(d_model=d_model, num_heads=8, feed_forward_dim=512, rate=0.25)(x)
    
    # Generate raw residuals
    residual_raw = tf.keras.layers.Dense(num_joints, activation='tanh')(x)
    
    # Smoothing
    # Stage 1: Light temporal smoothing (preserve detail)
    residual_smooth1 = tf.keras.layers.Conv1D(num_joints, 5, padding='same', activation='linear', 
                                             kernel_initializer='ones', use_bias=False)(residual_raw)
    residual_smooth1 = tf.keras.layers.Lambda(lambda x: x / 5.0)(residual_smooth1)  # Normalize
    
    # Stage 2: Medium temporal smoothing (reduce jitter)
    residual_smooth2 = tf.keras.layers.Conv1D(num_joints, 9, padding='same', activation='linear',
                                             kernel_initializer='ones', use_bias=False)(residual_raw)
    residual_smooth2 = tf.keras.layers.Lambda(lambda x: x / 9.0)(residual_smooth2)  # Normalize
    
    # Stage 3: Heavy temporal smoothing (eliminate rapid changes)
    residual_smooth3 = tf.keras.layers.Conv1D(num_joints, 15, padding='same', activation='linear',
                                             kernel_initializer='ones', use_bias=False)(residual_raw)
    residual_smooth3 = tf.keras.layers.Lambda(lambda x: x / 15.0)(residual_smooth3)  # Normalize
    
    # Adaptive blending: 50% raw (for creativity), 30% light smooth, 15% medium, 5% heavy
    residuals = tf.keras.layers.Lambda(lambda inputs: 
        0.5 * inputs[0] + 0.3 * inputs[1] + 0.15 * inputs[2] + 0.05 * inputs[3]
    )([residual_raw, residual_smooth1, residual_smooth2, residual_smooth3])
    
    # Additional velocity-based smoothing to prevent rapid direction changes
    def velocity_smoothing(residuals_input):
        # Calculate velocities
        velocities = residuals_input[:, 1:] - residuals_input[:, :-1]
        
        # Smooth velocities with smaller kernel
        velocities_smooth = tf.keras.layers.Conv1D(num_joints, 3, padding='same', 
                                                  activation='linear', use_bias=False)(
            tf.pad(velocities, [[0, 0], [0, 1], [0, 0]], mode='CONSTANT')
        )
        
        # Reconstruct positions from smoothed velocities
        velocities_smooth = velocities_smooth[:, :-1]  
        
        # Integrate back to positions
        first_position = residuals_input[:, 0:1]  # Keep first position
        positions = tf.concat([first_position, 
                              first_position + tf.cumsum(velocities_smooth, axis=1)], axis=1)
        return positions
    
    residuals = tf.keras.layers.Lambda(velocity_smoothing)(residuals)
    
    # Scale residuals 
    scaled_residuals = tf.keras.layers.Lambda(lambda x: x * 1.0)(residuals)
    
    # Bound residuals but with smoother clipping to avoid discontinuities
    def smooth_clipping(x, min_val=-1.0, max_val=1.0):
        return min_val + (max_val - min_val) * tf.sigmoid(x / 0.5)
    
    bounded_residuals = tf.keras.layers.Lambda(lambda x: smooth_clipping(x))(scaled_residuals)
    
    return tf.keras.Model([robot_sequence, style_sequence], bounded_residuals, name='temporal_transformer')

def build_robot_encoder(embed_dim=512):
    robot_input = tf.keras.Input(shape=(128, 7))
    x = tf.keras.layers.LSTM(128, return_sequences=True)(robot_input)
    x = tf.keras.layers.LSTM(embed_dim)(x)
    return tf.keras.Model(robot_input, x, name='robot_encoder')

def build_discriminator():
    motion_input = tf.keras.Input(shape=(128, 7))
    x = tf.keras.layers.LSTM(128, return_sequences=True)(motion_input)
    x = tf.keras.layers.LSTM(64)(x)
    x = tf.keras.layers.Dense(128, activation='relu')(x)
    output = tf.keras.layers.Dense(1, activation='sigmoid')(x)
    return tf.keras.Model(motion_input, output, name='discriminator')

def build_complete_temporal_model(use_cross_attention=True):
    
    # Components with configurable attention
    temporal_style_encoder = build_temporal_style_encoder(use_cross_attention=use_cross_attention)
    robot_encoder = build_robot_encoder()
    transformer = build_temporal_style_transfer_transformer()
    discriminator = build_discriminator() 
    
    # Training model inputs (fixed 128 length for training)
    video_input = tf.keras.Input(shape=(128, 64, 64, 3), name='video')
    pose_input = tf.keras.Input(shape=(128, 132), name='poses')
    robot_input = tf.keras.Input(shape=(128, 7), name='robot_motion')
    
    # Encode temporal style features 
    style_sequence = temporal_style_encoder([video_input, pose_input])
    
    # Encode robot motion
    robot_encoding = robot_encoder(robot_input)
    
    # Generate temporal residuals directly
    style_residuals = transformer([robot_input, style_sequence])
    
    # Apply residuals to create styled motion
    styled_motion = tf.keras.layers.Add()([robot_input, style_residuals])
    
    # Discriminator evaluation
    disc_score = discriminator(styled_motion)
    
    # Training model
    training_model = tf.keras.Model(
        inputs=[video_input, pose_input, robot_input],
        outputs={
            'styled_motion': styled_motion,
            'style_residual': style_residuals,
            'style_sequence': style_sequence,
            'robot_encoding': robot_encoding,
            'discriminator_score': disc_score
        }
    )
    
    components = {
        'temporal_style_encoder': temporal_style_encoder,
        'robot_encoder': robot_encoder,
        'transformer': transformer,
        'discriminator': discriminator
    }
    
    return training_model, discriminator, components

## Loss Functions

In [ ]:

def compute_temporal_variation_loss(residuals):
    
    # Calculate temporal variance of residuals
    def calculate_variance(x):
        return tf.reduce_mean(tf.math.reduce_variance(x, axis=1))
    
    residual_variance = calculate_variance(residuals)
    
    # Encourage minimum variance (negative loss to maximize)
    min_variance_loss = tf.maximum(0.0, 0.05 - residual_variance)
    
    # Calculate smoothness of residual changes
    def calculate_jerk(x):
        residual_velocity = x[:, 1:] - x[:, :-1]
        residual_acceleration = residual_velocity[:, 1:] - residual_velocity[:, :-1]
        return tf.reduce_mean(tf.square(residual_acceleration))
    
    jerk_penalty = calculate_jerk(residuals)
    
    return {
        'temporal_variance': residual_variance,
        'min_variance_loss': min_variance_loss,
        'jerk_penalty': jerk_penalty
    }

def compute_style_rhythm_loss(styled_motion, original_motion, style_sequence):
    
    # Extract velocity from both motions
    def extract_velocity(motion):
        return motion[:, 1:] - motion[:, :-1]
    
    styled_velocity = extract_velocity(styled_motion)
    original_velocity = extract_velocity(original_motion)
    
    # Calculate rhythm difference
    def calculate_rhythm(velocity):
        return tf.reduce_mean(tf.abs(velocity), axis=-1)
    
    styled_rhythm = calculate_rhythm(styled_velocity)
    original_rhythm = calculate_rhythm(original_velocity)
    
    # Encourage rhythm to deviate from original
    rhythm_deviation = tf.reduce_mean(tf.square(styled_rhythm - original_rhythm))
    
    # Encourage rhythm to correlate with style strength
    style_strength = tf.reduce_mean(tf.abs(style_sequence), axis=-1)
    style_strength_padded = style_strength[:, :-1]  # Match velocity length
    
    # Positive correlation between style strength and rhythm changes
    rhythm_style_correlation = tf.reduce_mean(styled_rhythm * style_strength_padded)
    
    return {
        'rhythm_deviation': rhythm_deviation,
        'rhythm_style_correlation': rhythm_style_correlation
    }

def compute_enhanced_losses(outputs, original_robot_motion, style_sequence, style_labels=None):
    
    styled_motion = outputs['styled_motion']
    style_residual = outputs['style_residual']
    
    losses = {}
    
    # Basic losses
    losses['adversarial'] = tf.reduce_mean(tf.square(outputs['discriminator_score'] - 1.0))
    
    # Temporal variation losses
    temporal_losses = compute_temporal_variation_loss(style_residual)
    losses.update(temporal_losses)
    
    # Style rhythm losses
    rhythm_losses = compute_style_rhythm_loss(styled_motion, original_robot_motion, style_sequence)
    losses.update(rhythm_losses)
    
    
    # 1. Velocity smoothness (first derivative)
    styled_velocity = styled_motion[:, 1:] - styled_motion[:, :-1]
    original_velocity = original_robot_motion[:, 1:] - original_robot_motion[:, :-1]
    
    velocity_smoothness = tf.reduce_mean(tf.square(styled_velocity))
    original_velocity_smoothness = tf.reduce_mean(tf.square(original_velocity))
    losses['velocity_smoothness'] = tf.maximum(0.0, velocity_smoothness - original_velocity_smoothness * 1.2)
    
    # 2. Acceleration smoothness (second derivative) - Key for reducing jitter
    styled_acceleration = styled_velocity[:, 1:] - styled_velocity[:, :-1]
    original_acceleration = original_velocity[:, 1:] - original_velocity[:, :-1]
    
    acceleration_smoothness = tf.reduce_mean(tf.square(styled_acceleration))
    original_acceleration_smoothness = tf.reduce_mean(tf.square(original_acceleration))
    losses['acceleration_smoothness'] = tf.maximum(0.0, acceleration_smoothness - original_acceleration_smoothness * 1.2)
    
    # 3. Jerk penalty (third derivative) - Critical for eliminating jittery motion
    styled_jerk = styled_acceleration[:, 1:] - styled_acceleration[:, :-1]
    original_jerk = original_acceleration[:, 1:] - original_acceleration[:, :-1]
    
    jerk_magnitude = tf.reduce_mean(tf.square(styled_jerk))
    original_jerk_magnitude = tf.reduce_mean(tf.square(original_jerk))
    
    # Strong penalty for excessive jerk
    losses['jerk_smoothness'] = tf.maximum(0.0, jerk_magnitude - original_jerk_magnitude * 1.1)
    
    # 4. High-frequency penalty - penalise rapid oscillations
    def high_frequency_penalty(motion):
        # Calculate second-order differences to detect high-frequency changes
        diff1 = motion[:, 1:] - motion[:, :-1]
        diff2 = diff1[:, 1:] - diff1[:, :-1]
        diff3 = diff2[:, 1:] - diff2[:, :-1]
        
        sign_changes = tf.reduce_mean(tf.abs(tf.sign(diff3[:, 1:]) - tf.sign(diff3[:, :-1])))
        return sign_changes
    
    styled_hf_penalty = high_frequency_penalty(styled_motion)
    original_hf_penalty = high_frequency_penalty(original_robot_motion)
    losses['high_frequency_penalty'] = tf.maximum(0.0, styled_hf_penalty - original_hf_penalty * 1.1)
    
    # Standard motion quality losses 
    motion_diff = styled_motion[:, 1:] - styled_motion[:, :-1]
    original_diff = original_robot_motion[:, 1:] - original_robot_motion[:, :-1]
    
    smoothness_loss = tf.reduce_mean(tf.square(motion_diff))
    original_smoothness = tf.reduce_mean(tf.square(original_diff))
    losses['smoothness'] = tf.maximum(0.0, smoothness_loss - original_smoothness * 2.0)
    
    # Position constraint (allow deviation but prevent extreme jumps)
    position_deviation = tf.reduce_mean(tf.square(styled_motion - original_robot_motion))
    losses['position_constraint'] = position_deviation
    
    # Residual penalties (encourage smooth residuals)
    losses['residual_penalty'] = tf.reduce_mean(tf.square(style_residual))
    
    # Residual smoothness penalty
    residual_velocity = style_residual[:, 1:] - style_residual[:, :-1]
    residual_acceleration = residual_velocity[:, 1:] - residual_velocity[:, :-1]
    losses['residual_smoothness'] = tf.reduce_mean(tf.square(residual_acceleration))
    
    # Minimum change requirement 
    motion_change = tf.reduce_mean(tf.abs(styled_motion - original_robot_motion))
    losses['minimum_change'] = tf.maximum(0.0, 0.15 - motion_change)  # Slightly reduced to balance with smoothness
    
    # Style differentiation
    if style_labels is not None and len(set(style_labels)) > 1:
        batch_size = tf.shape(styled_motion)[0]
        diff_sum = 0.0
        count = 0
        
        for i in range(min(3, batch_size)):
            for j in range(i+1, min(3, batch_size)):
                motion_distance = tf.reduce_mean(tf.square(styled_motion[i] - styled_motion[j]))
                diff_sum += tf.maximum(0.0, 0.1 - motion_distance)
                count += 1
        
        losses['style_differentiation'] = diff_sum / tf.maximum(1.0, tf.cast(count, tf.float32))
        losses['style_consistency'] = 0.0
    else:
        losses['style_consistency'] = 0.0
        losses['style_differentiation'] = 0.0
    
    total_loss = (
        0.1 * losses['adversarial'] +                    # Reduced
        1.0 * losses['velocity_smoothness'] +            # Velocity smoothness
        3.0 * losses['acceleration_smoothness'] +        # Acceleration smoothness (key for jitter)
        5.0 * losses['jerk_smoothness'] +                # Jerk penalty 
        2.0 * losses['high_frequency_penalty'] +         # High-frequency penalty
        1.0 * losses['residual_smoothness'] +            # Smooth residuals
        0.001 * losses['smoothness'] +                   # Smoothness 
        0.005 * losses['residual_penalty'] +             
        10.0 * losses['min_variance_loss'] +             # Keep temporal variation
        4.0 * losses['rhythm_deviation'] +               # Keep rhythm changes
        2.0 * losses['rhythm_style_correlation'] +       # Keep style correlation
        3.0 * losses['minimum_change'] +                 # Force style transfer
        0.02 * losses['position_constraint'] +           
        0.1 * losses['style_consistency'] +
        0.3 * losses['style_differentiation']
    )
    
    losses['total'] = total_loss
    return losses

def compute_discriminator_loss(discriminator, real_motions, fake_motions):
    real_scores = discriminator(real_motions)
    fake_scores = discriminator(fake_motions)
    real_loss = tf.reduce_mean(tf.square(real_scores - 1.0))
    fake_loss = tf.reduce_mean(tf.square(fake_scores))
    return (real_loss + fake_loss) / 2


## Training

In [ ]:

def load_video_and_poses(video_path, pose_path, target_size=(64, 64), sequence_length=128):
    try:
        video_frames = load_video(video_path, target_size, sequence_length)
        poses = np.load(pose_path).astype(np.float32)
        
        if len(video_frames) != len(poses):
            min_length = min(len(video_frames), len(poses))
            video_frames = video_frames[:min_length]
            poses = poses[:min_length]
        
        return video_frames.astype(np.float32), poses.astype(np.float32)
    except:
        return None, None

def organize_human_videos_with_poses():
    mapping_path = "robot_motion_data/preprocessed_poses/video_pose_mapping.pkl"
    
    if not os.path.exists(mapping_path):
        return {}
    
    with open(mapping_path, 'rb') as f:
        mapping = pickle.load(f)
    
    validated_mapping = {}
    for style, pairs in mapping.items():
        validated_pairs = []
        for video_path, pose_path in pairs:
            if os.path.exists(video_path) and os.path.exists(pose_path):
                validated_pairs.append((video_path, pose_path))
        if validated_pairs:
            validated_mapping[style] = validated_pairs
    
    return validated_mapping

def create_mixed_style_task(robot_sequences, video_pose_mapping):
    available_styles = list(video_pose_mapping.keys())
    if len(available_styles) < 2:
        # If only one style, create single-style task
        style = available_styles[0]
        style_pairs = video_pose_mapping[style]
        
        task_data = {'videos': [], 'poses': [], 'robot_motions': [], 'style_labels': []}
        
        for _ in range(3):  # 3 examples from same style
            video_path, pose_path = style_pairs[np.random.randint(len(style_pairs))]
            video_frames, poses = load_video_and_poses(video_path, pose_path)
            if video_frames is not None and poses is not None:
                robot_motion = robot_sequences[np.random.randint(len(robot_sequences))]
                
                task_data['videos'].append(video_frames)
                task_data['poses'].append(poses)
                task_data['robot_motions'].append(robot_motion)
                task_data['style_labels'].append(style)
    else:
        # Mixed style task
        task_data = {'videos': [], 'poses': [], 'robot_motions': [], 'style_labels': []}
        
        # Get one example from 3 different styles
        selected_styles = np.random.choice(available_styles, size=min(3, len(available_styles)), replace=False)
        
        for style in selected_styles:
            style_pairs = video_pose_mapping[style]
            video_path, pose_path = style_pairs[np.random.randint(len(style_pairs))]
            video_frames, poses = load_video_and_poses(video_path, pose_path)
            if video_frames is not None and poses is not None:
                robot_motion = robot_sequences[np.random.randint(len(robot_sequences))]
                
                task_data['videos'].append(video_frames)
                task_data['poses'].append(poses)
                task_data['robot_motions'].append(robot_motion)
                task_data['style_labels'].append(style)
    
    # Convert to numpy arrays
    if task_data['videos']:
        task_data['videos'] = np.array(task_data['videos'], dtype=np.float32)
        task_data['poses'] = np.array(task_data['poses'], dtype=np.float32)
        task_data['robot_motions'] = np.array(task_data['robot_motions'], dtype=np.float32)
    
    return task_data

def train_temporal_style_model(num_epochs=50, use_cross_attention=True):
    
    # Load data
    robot_sequences = np.load("processed_robot_sequences.npy").astype(np.float32)
    video_pose_mapping = organize_human_videos_with_poses()
    
    if not video_pose_mapping:
        print("No video-pose mapping found! Run preprocess_videos() first.")
        return None
    
    # Build model with configurable attention
    model, discriminator, components = build_complete_temporal_model(use_cross_attention=use_cross_attention)
    
    attention_type = "with cross-modal attention" if use_cross_attention else "without cross-modal attention"
    print(f"Building temporal style transfer model {attention_type}")
    
    # Optimizers
    gen_optimizer = tf.keras.optimizers.Adam(0.0003)
    disc_optimizer = tf.keras.optimizers.Adam(0.0003)
    
    print(f"Training temporal style transfer model with {len(robot_sequences)} robot sequences")
    
    for epoch in range(num_epochs):
        epoch_gen_loss = 0.0
        epoch_disc_loss = 0.0
        successful_tasks = 0
        
        for task_num in range(10):  # More tasks per epoch
            try:
                task = create_mixed_style_task(robot_sequences, video_pose_mapping)
                
                videos = tf.convert_to_tensor(task['videos'], dtype=tf.float32)
                poses = tf.convert_to_tensor(task['poses'], dtype=tf.float32)
                robot_motions = tf.convert_to_tensor(task['robot_motions'], dtype=tf.float32)
                style_labels = task['style_labels']
                
                # Train generator
                with tf.GradientTape() as gen_tape:
                    outputs = model([videos, poses, robot_motions], training=True)
                    losses = compute_enhanced_losses(outputs, robot_motions, outputs['style_sequence'], style_labels)
                    gen_loss = losses['total']
                
                gen_grads = gen_tape.gradient(gen_loss, model.trainable_variables)
                gen_optimizer.apply_gradients(zip(gen_grads, model.trainable_variables))
                
                # Train discriminator
                with tf.GradientTape() as disc_tape:
                    disc_loss = compute_discriminator_loss(discriminator, robot_motions, outputs['styled_motion'])
                
                disc_grads = disc_tape.gradient(disc_loss, discriminator.trainable_variables)
                disc_optimizer.apply_gradients(zip(disc_grads, discriminator.trainable_variables))
                
                epoch_gen_loss += gen_loss
                epoch_disc_loss += disc_loss
                successful_tasks += 1
                
            except Exception as e:
                print(f"Task {task_num + 1} failed: {e}")
                continue
        
        if successful_tasks > 0:
            avg_gen_loss = epoch_gen_loss / successful_tasks
            avg_disc_loss = epoch_disc_loss / successful_tasks
            
            if epoch % 10 == 0:
                print(f"Epoch {epoch}: Gen Loss: {avg_gen_loss:.4f}, Disc Loss: {avg_disc_loss:.4f}")
    
    # Save model with attention type in filename
    attention_suffix = "_cross_attention" if use_cross_attention else "_simple"
    model.save_weights(f"temporal_style_transfer_model{attention_suffix}.weights.h5")
    components['temporal_style_encoder'].save_weights(f"temporal_style_encoder{attention_suffix}.weights.h5")
    components['robot_encoder'].save_weights(f"robot_encoder{attention_suffix}.weights.h5")
    components['transformer'].save_weights(f"temporal_transformer{attention_suffix}.weights.h5")
    
    print(f"Temporal style transfer training completed {attention_type}!")
    return model, components

In [ ]:
train_temporal_style_model(num_epochs=50)

## Few shot adaptation

In [ ]:

def adapt_to_style_temporal(style_name, adaptation_steps=5, use_cross_attention=True):    
    model, discriminator, components = build_complete_temporal_model(use_cross_attention=use_cross_attention)
    
    attention_suffix = "_cross_attention" if use_cross_attention else "_simple"
    
    try:
        components['temporal_style_encoder'].load_weights(f"temporal_style_encoder{attention_suffix}.weights.h5")
        components['robot_encoder'].load_weights(f"robot_encoder{attention_suffix}.weights.h5")
        components['transformer'].load_weights(f"temporal_transformer{attention_suffix}.weights.h5")
        
        attention_type = "with cross-modal attention" if use_cross_attention else "without cross-modal attention"
        print(f"Loaded temporal model weights {attention_type} for adaptation to {style_name}")
    except Exception as e:
        print(f"No trained model found: {e}")
        return None
    
    # Load data
    robot_sequences = np.load("processed_robot_sequences.npy")
    video_pose_mapping = organize_human_videos_with_poses()
    
    if style_name not in video_pose_mapping:
        print(f"Style '{style_name}' not found!")
        return None
    
    # Load style examples
    style_pairs = video_pose_mapping[style_name]
    videos = []
    poses = []
    
    for video_path, pose_path in style_pairs[:6]:
        try:
            video_frames, pose_data = load_video_and_poses(video_path, pose_path)
            if video_frames is not None and pose_data is not None:
                videos.append(video_frames)
                poses.append(pose_data)
        except:
            continue
    
    if not videos:
        print("No valid examples found")
        return None
    
    videos = np.array(videos, dtype=np.float32)
    poses = np.array(poses, dtype=np.float32)
    motions = robot_sequences[:len(videos)]
    
    # Adaptation optimizer (lower learning rate)
    optimizer = tf.keras.optimizers.Adam(0.0001)
    
    for step in range(adaptation_steps):
        with tf.GradientTape() as tape:
            outputs = model([videos, poses, motions], training=True)
            style_labels = [style_name] * len(videos)
            
            # Use enhanced loss function for adaptation
            losses = compute_enhanced_losses(outputs, motions, outputs['style_sequence'], style_labels)
            
            # Focus on style-specific losses during adaptation
            adaptation_loss = (
                5.0 * losses['min_variance_loss'] +           # Force temporal variation
                3.0 * losses['rhythm_deviation'] +            # Encourage rhythm changes
                2.0 * losses['rhythm_style_correlation'] +    # Link rhythm to style
                1.0 * losses['style_differentiation'] +       # Differentiate from other styles
                0.5 * losses['jerk_smoothness'] +             # Prevent excessive jerkiness
                0.2 * losses['adversarial'] +                # Slight adversarial pressure
                0.1 * losses['residual_penalty'] +           # Minor residual penalty
                0.05 * losses['position_constraint']         # Minimal position constraint
            )
        
        gradients = tape.gradient(adaptation_loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))
        
        print(f"Adaptation step {step + 1}: Loss {adaptation_loss:.4f}")
    
    # Save adapted model
    components['temporal_style_encoder'].save_weights(f"adapted_{style_name}_temporal_encoder{attention_suffix}.weights.h5")
    components['robot_encoder'].save_weights(f"adapted_{style_name}_robot_encoder{attention_suffix}.weights.h5")
    components['transformer'].save_weights(f"adapted_{style_name}_temporal_transformer{attention_suffix}.weights.h5")
    
    attention_type = "with cross-modal attention" if use_cross_attention else "without cross-modal attention"
    print(f"Temporal model adaptation to '{style_name}' completed {attention_type}!")
    return model, components

In [ ]:
adapt_to_style_temporal("Flamenco", adaptation_steps=5)

## Generation

In [ ]:
def load_reference_poses(video_path):
    try:
        mapping_path = "robot_motion_data/preprocessed_poses/video_pose_mapping.pkl"
        if not os.path.exists(mapping_path):
            return None
        
        with open(mapping_path, 'rb') as f:
            mapping = pickle.load(f)
        
        for style, pairs in mapping.items():
            for vid_path, pose_path in pairs:
                if os.path.abspath(vid_path) == os.path.abspath(video_path):
                    poses = np.load(pose_path).astype(np.float32)
                    return poses
        
        return None
    except:
        return None

def generate_temporal_styled_motion(robot_csv_path, reference_video_path, style_name=None, creativity=1.0, use_cross_attention=True):
    
    # Load model components
    model, discriminator, components = build_complete_temporal_model(use_cross_attention=use_cross_attention)
    
    attention_suffix = "_cross_attention" if use_cross_attention else "_simple"
    
    # Load weights (try adapted version first if style_name provided)
    weight_files = []
    if style_name:
        weight_files.append((
            f"adapted_{style_name}_temporal_encoder{attention_suffix}.weights.h5",
            f"adapted_{style_name}_robot_encoder{attention_suffix}.weights.h5",
            f"adapted_{style_name}_temporal_transformer{attention_suffix}.weights.h5"
        ))
    
    # Fallback to general trained model
    weight_files.append((
        f"temporal_style_encoder{attention_suffix}.weights.h5",
        f"robot_encoder{attention_suffix}.weights.h5",
        f"temporal_transformer{attention_suffix}.weights.h5"
    ))
    
    loaded = False
    for encoder_weights, robot_weights, transformer_weights in weight_files:
        try:
            components['temporal_style_encoder'].load_weights(encoder_weights)
            components['robot_encoder'].load_weights(robot_weights)
            components['transformer'].load_weights(transformer_weights)
            loaded = True
            attention_type = "with cross-modal attention" if use_cross_attention else "without cross-modal attention"
            print(f"Loaded weights {attention_type}: {transformer_weights}")
            break
        except Exception as e:
            continue
    
    if not loaded:
        print("No trained weights found")
        return None
    
    # Load data
    with open("robot_scaler.pkl", 'rb') as f:
        scaler = pickle.load(f)
    
    robot_data = load_robot_csv(robot_csv_path)
    reference_video = load_video(reference_video_path)
    reference_poses = load_reference_poses(reference_video_path)
    
    if reference_poses is None:
        print("Could not load poses for reference video")
        return None
    
    # Scale robot data
    scaled_robot_data = scaler.transform(robot_data).astype(np.float32)
    
    # Extract temporal style features (now with optional cross-modal attention)
    video_tensor = tf.convert_to_tensor(np.expand_dims(reference_video, 0), dtype=tf.float32)
    pose_tensor = tf.convert_to_tensor(np.expand_dims(reference_poses, 0), dtype=tf.float32)
    style_sequence = components['temporal_style_encoder']([video_tensor, pose_tensor])[0]
    
    print(f"Robot data shape: {scaled_robot_data.shape}")
    print(f"Style sequence shape: {style_sequence.shape}")
    attention_type = "with cross-modal attention" if use_cross_attention else "without cross-modal attention"
    print(f"Using temporal style encoder {attention_type}")
    
    # Use overlapping windows with enhanced smooth blending
    sequence_length = 128
    window_step = 32  # Increased overlap (75% overlap instead of 50%)
    num_timesteps = len(scaled_robot_data)
    
    # Initialize styled motion with better interpolation
    styled_motion = np.zeros_like(scaled_robot_data)
    weight_accumulator = np.zeros(num_timesteps)
    
    for window_start in range(0, num_timesteps - sequence_length + 1, window_step):
        window_end = window_start + sequence_length
        
        # Prepare robot window
        robot_window = scaled_robot_data[window_start:window_end]
        
        # Prepare style window (cycle through style if needed)
        style_window = np.zeros((sequence_length, style_sequence.shape[-1]), dtype=np.float32)
        for i in range(sequence_length):
            style_idx = (window_start + i) % len(style_sequence)
            style_window[i] = style_sequence[style_idx]
        
        # Generate temporal residuals
        robot_tensor = tf.convert_to_tensor(np.expand_dims(robot_window, 0), dtype=tf.float32)
        style_tensor = tf.convert_to_tensor(np.expand_dims(style_window, 0), dtype=tf.float32)
        
        try:
            residuals = components['transformer']([robot_tensor, style_tensor])[0]
            
            # Convert TensorFlow tensor to numpy array
            residuals_np = residuals.numpy()
            
            # Debug information
            if window_start == 0:  # Print only for first window to avoid spam
                print(f"Residuals shape: {residuals_np.shape}, type: {type(residuals_np)}")
                print(f"Robot window shape: {robot_window.shape}, type: {type(robot_window)}")
            
            # Apply creativity scaling
            scaled_residuals = creativity * residuals_np
            
            # Create styled motion for this window
            styled_window = robot_window + scaled_residuals
            
            # Create enhanced blending weights (smoother transitions)
            blend_weights = np.ones(sequence_length)
            fade_length = 64  # Longer fade regions (50% of window)
            
            # Cosine fade-in at start (smoother than linear)
            fade_in = 0.5 * (1 - np.cos(np.pi * np.linspace(0, 1, fade_length)))
            blend_weights[:fade_length] = 0.1 + 0.9 * fade_in
            
            # Cosine fade-out at end
            fade_out = 0.5 * (1 + np.cos(np.pi * np.linspace(0, 1, fade_length)))
            blend_weights[-fade_length:] = 0.1 + 0.9 * fade_out
            
            # Apply temporal smoothing to the styled window itself
            for joint in range(7):
                joint_data = styled_window[:, joint]
                
                # Light smoothing with preserving boundaries
                if len(joint_data) >= 5:
                    # Use simple moving average in the middle, preserve edges
                    smoothed = joint_data.copy()  # Now joint_data is numpy array
                    for i in range(2, len(joint_data) - 2):
                        smoothed[i] = 0.2 * (joint_data[i-2] + joint_data[i-1] + joint_data[i] + 
                                           joint_data[i+1] + joint_data[i+2])
                    
                    # Blend original and smoothed (favor original for style preservation)
                    styled_window[:, joint] = 0.7 * joint_data + 0.3 * smoothed
            
            # Accumulate weighted results
            for i, global_idx in enumerate(range(window_start, window_end)):
                if global_idx < num_timesteps:
                    weight = blend_weights[i]
                    styled_motion[global_idx] += weight * styled_window[i]
                    weight_accumulator[global_idx] += weight
                    
        except Exception as e:
            print(f"Error processing window {window_start}-{window_end}: {e}")
            print(f"Error type: {type(e)}")
            import traceback
            traceback.print_exc()
            continue
    
    # Handle remaining timesteps at the end
    if window_start + sequence_length < num_timesteps:
        remaining_start = num_timesteps - sequence_length
        remaining_robot = scaled_robot_data[remaining_start:]
        
        # Pad to 128 length
        robot_window = np.zeros((sequence_length, 7), dtype=np.float32)
        robot_window[:len(remaining_robot)] = remaining_robot
        robot_window[len(remaining_robot):] = remaining_robot[-1:]
        
        # Style window
        style_window = np.zeros((sequence_length, style_sequence.shape[-1]), dtype=np.float32)
        for i in range(sequence_length):
            style_idx = (remaining_start + i) % len(style_sequence)
            style_window[i] = style_sequence[style_idx]
        
        try:
            robot_tensor = tf.convert_to_tensor(np.expand_dims(robot_window, 0), dtype=tf.float32)
            style_tensor = tf.convert_to_tensor(np.expand_dims(style_window, 0), dtype=tf.float32)
            
            residuals = components['transformer']([robot_tensor, style_tensor])[0]
            
            # Convert TensorFlow tensor to numpy array
            residuals_np = residuals.numpy()
            scaled_residuals = creativity * residuals_np[:len(remaining_robot)]
            
            styled_remaining = remaining_robot + scaled_residuals
            
            # Add remaining part
            for i, global_idx in enumerate(range(remaining_start, num_timesteps)):
                if weight_accumulator[global_idx] == 0:
                    styled_motion[global_idx] = styled_remaining[i]
                    weight_accumulator[global_idx] = 1.0
                    
        except Exception as e:
            print(f"Error processing remaining timesteps: {e}")
            # Fill remaining with original data
            for i, global_idx in enumerate(range(remaining_start, num_timesteps)):
                if weight_accumulator[global_idx] == 0:
                    styled_motion[global_idx] = remaining_robot[i]
                    weight_accumulator[global_idx] = 1.0
    
    # Normalize by accumulated weights and apply final light smoothing
    for i in range(num_timesteps):
        if weight_accumulator[i] > 0:
            styled_motion[i] /= weight_accumulator[i]
        else:
            styled_motion[i] = scaled_robot_data[i]  # Fallback to original
    
    # Final light smoothing to ensure continuity between blended regions
    for joint in range(7):
        joint_motion = styled_motion[:, joint]
        if len(joint_motion) >= 3:
            # Very light 3-point smoothing
            smoothed = joint_motion.copy()
            for i in range(1, len(joint_motion) - 1):
                smoothed[i] = 0.25 * joint_motion[i-1] + 0.5 * joint_motion[i] + 0.25 * joint_motion[i+1]
            
            # Minimal blending (95% original to preserve style)
            styled_motion[:, joint] = 0.95 * joint_motion + 0.05 * smoothed
    
    # Convert back to original scale
    final_motion = scaler.inverse_transform(styled_motion).astype(np.float32)
    
    # Save result
    base_name = os.path.splitext(os.path.basename(robot_csv_path))[0]
    style_suffix = f"_{style_name}" if style_name else ""
    attention_suffix_file = "_cross_attn" if use_cross_attention else "_simple"
    creativity_suffix = f"_c{creativity}_temporal{attention_suffix_file}"
    output_path = f"temporal_styled{style_suffix}{creativity_suffix}_{base_name}.csv"
    
    np.savetxt(output_path, final_motion, delimiter=',',
               header='Position1,Position2,Position3,Position4,Position5,Position6,Position7',
               comments='')
    
    print(f"Generated temporal styled motion saved to: {output_path}")
    
    
    # Save result
    base_name = os.path.splitext(os.path.basename(robot_csv_path))[0]
    style_suffix = f"_{style_name}" if style_name else ""
    creativity_suffix = f"_c{creativity}_temporal_smooth"
    output_path = f"temporal_styled{style_suffix}{creativity_suffix}_{base_name}.csv"
    
    np.savetxt(output_path, final_motion, delimiter=',',
               header='Position1,Position2,Position3,Position4,Position5,Position6,Position7',
               comments='')
    
    return final_motion

In [ ]:
generate_temporal_styled_motion(
    "robot_motion_data/robot_motions/example.csv", 
    "robot_motion_data/human_videos/Flamenco/example.mp4", 
    "Flamenco", 
    creativity=0.25, 
    use_cross_attention=True)

## Evaluation

In [ ]:
def evaluate_style_transfer_comprehensive(original_csv_path, styled_csv_path, reference_video_path=None):
    
    print(f"Evaluating style transfer comprehensively:")
    print(f"  Original: {os.path.basename(original_csv_path)}")
    print(f"  Styled: {os.path.basename(styled_csv_path)}")
    
    # Load robot motions
    try:
        original_robot = np.loadtxt(original_csv_path, delimiter=',', skiprows=1)
        styled_robot = np.loadtxt(styled_csv_path, delimiter=',', skiprows=1)
        
        print(f"Loaded original robot motion: {original_robot.shape}")
        print(f"Loaded styled robot motion: {styled_robot.shape}")
        
    except Exception as e:
        print(f"Error loading CSV files: {e}")
        return None
    
    # Ensure same dimensions
    min_length = min(len(original_robot), len(styled_robot))
    min_joints = min(original_robot.shape[1], styled_robot.shape[1])
    
    original_robot = original_robot[:min_length, :min_joints]
    styled_robot = styled_robot[:min_length, :min_joints]
    
    print(f"Analyzing {min_length} timesteps across {min_joints} joints")
    print(f"Final shapes - Original: {original_robot.shape}, Styled: {styled_robot.shape}")
    
    results = {}
    
    # 1. IMPROVED TRAJECTORY PRESERVATION
    trajectory_results = evaluate_trajectory_preservation_improved(original_robot, styled_robot)
    results['trajectory_analysis'] = trajectory_results
    
    # 2. PHYSICAL DEVIATION (from original evaluation)
    joint_deviations = np.std(styled_robot - original_robot, axis=0)
    avg_deviation = np.mean(joint_deviations)
    max_deviation = np.max(joint_deviations)
    
    results['physical_deviation'] = {
        'average': avg_deviation,
        'maximum': max_deviation,
        'per_joint': joint_deviations
    }
    
    # 3. MOTION CHARACTERISTICS ANALYSIS
    original_characteristics = analyze_motion_characteristics(original_robot)
    styled_characteristics = analyze_motion_characteristics(styled_robot)
    
    results['motion_profiles'] = {
        'original': original_characteristics,
        'styled': styled_characteristics
    }
    
    # 4. STYLE DIFFERENTIATION
    speed_change = abs(styled_characteristics['avg_speed'] - original_characteristics['avg_speed']) / max(original_characteristics['avg_speed'], 1e-6)
    accel_change = abs(styled_characteristics['avg_acceleration'] - original_characteristics['avg_acceleration']) / max(original_characteristics['avg_acceleration'], 1e-6)
    coordination_change = abs(styled_characteristics['coordination'] - original_characteristics['coordination'])
    range_change = abs(styled_characteristics['movement_range'] - original_characteristics['movement_range']) / max(original_characteristics['movement_range'], 1e-6)
    
    # Overall style change
    style_change_score = np.mean([speed_change, accel_change, coordination_change, range_change])
    
    results['style_characteristics'] = {
        'speed_change': speed_change,
        'acceleration_change': accel_change,
        'coordination_change': coordination_change,
        'range_change': range_change,
        'overall_style_change': style_change_score
    }
    
    # 5. SMOOTHNESS EVALUATION
    original_smoothness = compute_smoothness(original_robot)
    styled_smoothness = compute_smoothness(styled_robot)
    smoothness_ratio = styled_smoothness / max(original_smoothness, 1e-6)
    
    results['motion_quality'] = {
        'original_smoothness': original_smoothness,
        'styled_smoothness': styled_smoothness,
        'smoothness_ratio': smoothness_ratio
    }
    
    # 6. FREQUENCY ANALYSIS
    original_freq = analyze_frequency_content(original_robot)
    styled_freq = analyze_frequency_content(styled_robot)
    frequency_change = abs(styled_freq - original_freq) / max(original_freq, 1e-6)
    
    results['frequency_analysis'] = {
        'original_freq': original_freq,
        'styled_freq': styled_freq,
        'frequency_change': frequency_change
    }
    
    # 7.  OVERALL ASSESSMENT
    trajectory_score = trajectory_results['functional_preservation']['overall_score']
    deviation_score = max(0, 1 - avg_deviation / 2.0)
    style_score = min(1, style_change_score)
    
    overall_quality = (
        trajectory_score * 0.6 +    
        deviation_score * 0.2 +     
        style_score * 0.2           
    )
    
    results['overall_assessment'] = {
        'overall_quality': overall_quality,
        'component_scores': {
            'trajectory_score': trajectory_score,
            'deviation_score': deviation_score,
            'style_score': style_score
        }
    }
    
    return results

def analyze_temporal_dynamics(motion_sequence):
    # Detect rhythm patterns
    fft = np.fft.fft(motion_sequence)
    power_spectrum = np.abs(fft[:len(fft)//2])
    dominant_frequency = np.argmax(power_spectrum)
    rhythm_strength = np.max(power_spectrum) / np.mean(power_spectrum)
    
    # Analyze motion phases (acceleration vs deceleration)
    velocity = np.diff(motion_sequence)
    acceleration = np.diff(velocity)
    
    return {
        'dominant_frequency': dominant_frequency,
        'rhythm_strength': rhythm_strength,
        'phase_transitions': len(np.where(np.diff(np.sign(acceleration)))[0]),
        'motion_complexity': np.std(acceleration)
    }
def extract_video_style_features(video_path):
    try:
        cap = cv2.VideoCapture(video_path)
        frames = []
        
        # Read video frames
        while len(frames) < 128:  # Match training sequence length
            ret, frame = cap.read()
            if not ret:
                break
            frame = cv2.resize(frame, (64, 64))
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame = frame.astype(np.float32) / 255.0
            frames.append(frame)
        
        cap.release()
        
        # Pad if necessary
        while len(frames) < 128:
            frames.append(frames[-1])
        
        video_array = np.array(frames[:128])
        
        # Extract motion features (frame differences)
        motion_vectors = []
        for t in range(1, len(video_array)):
            motion = video_array[t] - video_array[t-1]
            motion_magnitude = np.mean(np.abs(motion))
            motion_vectors.append(motion_magnitude)
        
        motion_vectors.insert(0, motion_vectors[0])  # Pad first frame
        
        return {
            'motion_intensity': np.mean(motion_vectors),
            'motion_variance': np.var(motion_vectors),
            'motion_profile': motion_vectors,
            'temporal_dynamics': analyze_temporal_dynamics(motion_vectors)
        }
        
    except Exception as e:
        print(f"Error extracting video features: {e}")
        return None


def extract_pose_style_features(pose_path):
    try:
        poses = np.load(pose_path).astype(np.float32)
        
        # Reshape to (time, joints, coordinates) assuming 33 joints with 4 coords each
        poses_reshaped = poses.reshape(poses.shape[0], 33, 4)
        pose_coords = poses_reshaped[:, :, :3]  # Only x, y, z coordinates
        
        # Calculate pose dynamics
        pose_velocity = np.diff(pose_coords, axis=0)
        pose_acceleration = np.diff(pose_velocity, axis=0)
        
        # Extract rhythm and coordination features
        velocity_magnitude = np.mean(np.abs(pose_velocity), axis=(1, 2))
        acceleration_magnitude = np.mean(np.abs(pose_acceleration), axis=(1, 2))
        
        # Joint coordination patterns (correlation between different joints)
        coordination_features = []
        for i in range(0, 33, 4):  # Sample every 4th joint
            if i < pose_coords.shape[1]:
                joint_motion = pose_coords[:, i, :]
                joint_velocity = np.mean(np.abs(np.diff(joint_motion, axis=0)), axis=1)
                coordination_features.append(np.mean(joint_velocity))
        
        return {
            'rhythm_intensity': np.mean(velocity_magnitude),
            'rhythm_variance': np.var(velocity_magnitude),
            'acceleration_intensity': np.mean(acceleration_magnitude),
            'acceleration_variance': np.var(acceleration_magnitude),
            'coordination_score': np.mean(coordination_features),
            'movement_range': np.mean(np.ptp(pose_coords, axis=0)),
            'temporal_consistency': analyze_pose_temporal_patterns(pose_coords)
        }
        
    except Exception as e:
        print(f"Error extracting pose features: {e}")
        return None

def extract_robot_motion_style_features(robot_motion):    
    # Calculate motion dynamics
    velocity = np.diff(robot_motion, axis=0)
    acceleration = np.diff(velocity, axis=0)
    jerk = np.diff(acceleration, axis=0)
    
    # Speed and rhythm characteristics
    speed = np.linalg.norm(velocity, axis=1)
    accel_magnitude = np.linalg.norm(acceleration, axis=1)
    
    # Joint coordination
    joint_correlations = []
    for i in range(robot_motion.shape[1]):
        for j in range(i+1, robot_motion.shape[1]):
            try:
                corr, _ = pearsonr(robot_motion[:, i], robot_motion[:, j])
                if not np.isnan(corr):
                    joint_correlations.append(abs(corr))
            except:
                continue
    
    coordination_score = np.mean(joint_correlations) if joint_correlations else 0
    
    # Movement range and spatial characteristics
    movement_range = np.ptp(robot_motion, axis=0).mean()
    
    # Temporal dynamics (similar to video analysis)
    temporal_dynamics = analyze_temporal_dynamics(speed)
    
    return {
        'motion_intensity': np.mean(speed),
        'motion_variance': np.var(speed),
        'rhythm_intensity': np.mean(speed),
        'rhythm_variance': np.var(speed),
        'acceleration_intensity': np.mean(accel_magnitude),
        'acceleration_variance': np.var(accel_magnitude),
        'coordination_score': coordination_score,
        'movement_range': movement_range,
        'temporal_consistency': {
            'movement_fluidity': 1.0 / (1.0 + np.std(speed)),
            'acceleration_ratio': len(np.where(np.diff(speed) > 0)[0]) / max(1, len(speed)-1),
            'spatial_coverage': np.std(robot_motion, axis=0).mean(),
            'temporal_stability': 1.0 / (1.0 + np.std(np.diff(speed)))
        },
        'temporal_dynamics': temporal_dynamics
    }

def load_reference_style_data(style_name, video_pose_mapping_path="robot_motion_data/preprocessed_poses/video_pose_mapping.pkl"):
    try:
        if not os.path.exists(video_pose_mapping_path):
            print(f"Warning: Video-pose mapping file not found: {video_pose_mapping_path}")
            return None, None
        
        with open(video_pose_mapping_path, 'rb') as f:
            mapping = pickle.load(f)
        
        if style_name not in mapping:
            print(f"Warning: Style '{style_name}' not found in mapping. Available styles: {list(mapping.keys())}")
            return None, None
        
        style_pairs = mapping[style_name]
        
        # Analyse multiple examples if available
        video_features_list = []
        pose_features_list = []
        
        for video_path, pose_path in style_pairs[:3]:  
            if os.path.exists(video_path) and os.path.exists(pose_path):
                video_feat = extract_video_style_features(video_path)
                pose_feat = extract_pose_style_features(pose_path)
                
                if video_feat:
                    video_features_list.append(video_feat)
                if pose_feat:
                    pose_features_list.append(pose_feat)
        
        # Average features across examples
        if video_features_list:
            avg_video_features = {}
            for key in video_features_list[0].keys():
                if key != 'motion_profile': 
                    if isinstance(video_features_list[0][key], dict):
                        avg_video_features[key] = {}
                        for subkey in video_features_list[0][key].keys():
                            values = [vf[key][subkey] for vf in video_features_list if subkey in vf[key]]
                            avg_video_features[key][subkey] = np.mean(values) if values else 0
                    else:
                        values = [vf[key] for vf in video_features_list]
                        avg_video_features[key] = np.mean(values)
        else:
            avg_video_features = None
        
        if pose_features_list:
            avg_pose_features = {}
            for key in pose_features_list[0].keys():
                if isinstance(pose_features_list[0][key], dict):
                    avg_pose_features[key] = {}
                    for subkey in pose_features_list[0][key].keys():
                        values = [pf[key][subkey] for pf in pose_features_list if subkey in pf[key]]
                        avg_pose_features[key][subkey] = np.mean(values) if values else 0
                else:
                    values = [pf[key] for pf in pose_features_list]
                    avg_pose_features[key] = np.mean(values)
        else:
            avg_pose_features = None
        
        return avg_video_features, avg_pose_features
        
    except Exception as e:
        print(f"Error loading reference style data: {e}")
        return None, None

def compare_style_features(robot_features, video_features=None, pose_features=None, style_name="unknown"):
    
    style_similarity_results = {
        'style_name': style_name,
        'video_similarity': {},
        'pose_similarity': {},
        'overall_style_match': 0.0
    }
    
    similarities = []
    
    if video_features:
        video_sim = {}
        
        # Motion intensity similarity
        motion_sim = 1.0 - abs(robot_features['motion_intensity'] - video_features['motion_intensity']) / \
                     max(robot_features['motion_intensity'], video_features['motion_intensity'], 1e-6)
        video_sim['motion_intensity'] = max(0, motion_sim)
        
        # Motion variance similarity
        variance_sim = 1.0 - abs(robot_features['motion_variance'] - video_features['motion_variance']) / \
                      max(robot_features['motion_variance'], video_features['motion_variance'], 1e-6)
        video_sim['motion_variance'] = max(0, variance_sim)
        
        # Temporal dynamics similarity
        if 'temporal_dynamics' in video_features:
            rhythm_sim = 1.0 - abs(robot_features['temporal_dynamics']['rhythm_strength'] - 
                                 video_features['temporal_dynamics']['rhythm_strength']) / \
                         max(robot_features['temporal_dynamics']['rhythm_strength'], 
                            video_features['temporal_dynamics']['rhythm_strength'], 1e-6)
            video_sim['rhythm_similarity'] = max(0, rhythm_sim)
            similarities.append(rhythm_sim)
        
        style_similarity_results['video_similarity'] = video_sim
        similarities.extend([motion_sim, variance_sim])
    
    if pose_features:
        pose_sim = {}
        
        # Rhythm intensity similarity
        rhythm_sim = 1.0 - abs(robot_features['rhythm_intensity'] - pose_features['rhythm_intensity']) / \
                     max(robot_features['rhythm_intensity'], pose_features['rhythm_intensity'], 1e-6)
        pose_sim['rhythm_intensity'] = max(0, rhythm_sim)
        
        # Acceleration similarity
        accel_sim = 1.0 - abs(robot_features['acceleration_intensity'] - pose_features['acceleration_intensity']) / \
                    max(robot_features['acceleration_intensity'], pose_features['acceleration_intensity'], 1e-6)
        pose_sim['acceleration_intensity'] = max(0, accel_sim)
        
        # Coordination similarity
        coord_sim = 1.0 - abs(robot_features['coordination_score'] - pose_features['coordination_score']) / \
                    max(robot_features['coordination_score'], pose_features['coordination_score'], 1e-6)
        pose_sim['coordination'] = max(0, coord_sim)
        
        # Movement range similarity
        range_sim = 1.0 - abs(robot_features['movement_range'] - pose_features['movement_range']) / \
                    max(robot_features['movement_range'], pose_features['movement_range'], 1e-6)
        pose_sim['movement_range'] = max(0, range_sim)
        
        # Temporal consistency similarity
        if 'temporal_consistency' in pose_features:
            fluidity_sim = 1.0 - abs(robot_features['temporal_consistency']['movement_fluidity'] - 
                                   pose_features['temporal_consistency']['movement_fluidity'])
            pose_sim['fluidity'] = max(0, fluidity_sim)
            similarities.append(fluidity_sim)
        
        style_similarity_results['pose_similarity'] = pose_sim
        similarities.extend([rhythm_sim, accel_sim, coord_sim, range_sim])
    
    # Calculate overall style match
    if similarities:
        style_similarity_results['overall_style_match'] = np.mean(similarities)
    
    return style_similarity_results

def evaluate_style_transfer_with_reference(original_csv_path, styled_csv_path, style_name=None, reference_video_path=None):
    print(f"Evaluating style transfer with reference style comparison:")
    print(f"  Original: {os.path.basename(original_csv_path)}")
    print(f"  Styled: {os.path.basename(styled_csv_path)}")
    print(f"  Target style: {style_name if style_name else 'Unknown'}")
    
    base_results = evaluate_style_transfer_comprehensive(original_csv_path, styled_csv_path)
    
    if not base_results:
        return None
    
    # Load robot motions
    try:
        original_robot = np.loadtxt(original_csv_path, delimiter=',', skiprows=1)
        styled_robot = np.loadtxt(styled_csv_path, delimiter=',', skiprows=1)
    except Exception as e:
        print(f"Error loading robot motions: {e}")
        return base_results
    
    # Ensure same dimensions
    min_length = min(len(original_robot), len(styled_robot))
    min_joints = min(original_robot.shape[1], styled_robot.shape[1])
    original_robot = original_robot[:min_length, :min_joints]
    styled_robot = styled_robot[:min_length, :min_joints]
    
    # Extract style features from robot motions
    original_robot_features = extract_robot_motion_style_features(original_robot)
    styled_robot_features = extract_robot_motion_style_features(styled_robot)
    
    # Load reference style data
    reference_video_features = None
    reference_pose_features = None
    
    if style_name:
        print(f"Loading reference data for style: {style_name}")
        reference_video_features, reference_pose_features = load_reference_style_data(style_name)
        
        if reference_video_features or reference_pose_features:
            print("✓ Reference style data loaded successfully")
        else:
            print("⚠ Could not load reference style data")
    
    # Compare styled robot with reference style
    style_comparison = compare_style_features(
        styled_robot_features, 
        reference_video_features, 
        reference_pose_features, 
        style_name
    )
    
    # Compare original robot with reference style (baseline)
    original_style_comparison = compare_style_features(
        original_robot_features,
        reference_video_features,
        reference_pose_features,
        style_name
    )
    
    # Add style comparison results to base results
    base_results['style_feature_analysis'] = {
        'styled_robot_features': styled_robot_features,
        'original_robot_features': original_robot_features,
        'reference_video_features': reference_video_features,
        'reference_pose_features': reference_pose_features,
        'styled_vs_reference': style_comparison,
        'original_vs_reference': original_style_comparison,
        'style_transfer_improvement': style_comparison['overall_style_match'] - original_style_comparison['overall_style_match']
    }
    
    return base_results

def print_enhanced_evaluation(results):
    print_comprehensive_evaluation(results)
    
    # Add style feature analysis
    if 'style_feature_analysis' in results:
        style_analysis = results['style_feature_analysis']
        
        print("\n" + "="*80)
        print(" STYLE FEATURE ANALYSIS - REFERENCE COMPARISON")
        print("="*80)
        
        styled_comparison = style_analysis['styled_vs_reference']
        original_comparison = style_analysis['original_vs_reference']
        improvement = style_analysis['style_transfer_improvement']
        
        print(f"\n Target Style: {styled_comparison['style_name']}")
        print(f" Style Match Score: {styled_comparison['overall_style_match']:.3f}")
        print(f" Baseline (Original): {original_comparison['overall_style_match']:.3f}")
        print(f" Style Transfer Improvement: {improvement:.3f}")
        
        if improvement > 0.2:
            print("  EXCELLENT - Significant style characteristics captured")
        elif improvement > 0.1:
            print("  GOOD - Noticeable style characteristics captured")
        elif improvement > 0.05:
            print("  MODERATE - Some style characteristics captured")
        else:
            print("  MINIMAL - Limited style characteristics captured")
        
        # Video similarity details
        if styled_comparison['video_similarity']:
            print(f"\n Video Style Similarity:")
            video_sim = styled_comparison['video_similarity']
            for feature, score in video_sim.items():
                print(f"   {feature.replace('_', ' ').title()}: {score:.3f}")
        
        # Pose similarity details
        if styled_comparison['pose_similarity']:
            print(f"\n Pose Style Similarity:")
            pose_sim = styled_comparison['pose_similarity']
            for feature, score in pose_sim.items():
                print(f"   {feature.replace('_', ' ').title()}: {score:.3f}")
        
        # Reference vs styled feature comparison
        if style_analysis['reference_video_features'] and style_analysis['reference_pose_features']:
            print(f"\n Style Characteristic Comparison:")
            styled_features = style_analysis['styled_robot_features']
            ref_video = style_analysis['reference_video_features']
            ref_pose = style_analysis['reference_pose_features']
            
            print(f"   Motion Intensity - Robot: {styled_features['motion_intensity']:.3f}, Video: {ref_video['motion_intensity']:.3f}")
            print(f"   Rhythm Intensity - Robot: {styled_features['rhythm_intensity']:.3f}, Pose: {ref_pose['rhythm_intensity']:.3f}")
            print(f"   Coordination - Robot: {styled_features['coordination_score']:.3f}, Pose: {ref_pose['coordination_score']:.3f}")
            print(f"   Movement Range - Robot: {styled_features['movement_range']:.3f}, Pose: {ref_pose['movement_range']:.3f}")
        
        print("="*80)


def print_comprehensive_evaluation(results):    
    print("\n" + "="*80)
    print(" COMPREHENSIVE ROBOTIC STYLE TRANSFER EVALUATION")
    print("="*80)
    
    # Overall Quality
    overall = results['overall_assessment']['overall_quality']
    print(f"\n Overall Quality Score: {overall:.3f}")
    if overall > 0.8:
        print("  EXCELLENT - Great style transfer!")
    elif overall > 0.6:
        print("  GOOD - Effective style transfer") 
    elif overall > 0.4:
        print("   FAIR - Some style transfer detected")
    else:
        print("  POOR - Limited effective style transfer")
    
    # Trajectory Analysis
    trajectory = results['trajectory_analysis']
    func_score = trajectory['functional_preservation']['overall_score']
    print(f"\n Functional Preservation: {func_score:.3f}")
    if func_score > 0.8:
        print("  EXCELLENT - Robot maintains excellent functionality")
    elif func_score > 0.6:
        print("  GOOD - Robot maintains good functionality")
    elif func_score > 0.4:
        print("   FAIR - Robot has moderate functional issues")
    else:
        print("  POOR - Robot has significant functional issues")
    
    # trajectory metrics
    acc = trajectory['absolute_accuracy']
    print(f"\n   Position Accuracy:")
    print(f"     Average MAE: {acc['average_mae']:.3f} radians")
    print(f"     Max Error: {acc['max_error_across_joints']:.3f} radians")
    
    if acc['average_mae'] < 0.1:
        print("     EXCELLENT - Very accurate positioning")
    elif acc['average_mae'] < 0.3:
        print("     GOOD - Acceptable positioning")
    elif acc['average_mae'] < 0.5:
        print("     FAIR - Moderate positioning errors")
    else:
        print("     POOR - Significant positioning errors")
    
    pattern = trajectory['pattern_similarity']
    print(f"\n   Pattern Similarity: {pattern['average_correlation']:.3f}")
    if pattern['average_correlation'] > 0.8:
        print("     EXCELLENT - Motion patterns very similar")
    elif pattern['average_correlation'] > 0.6:
        print("     GOOD - Motion patterns mostly similar")
    else:
        print("     NEEDS IMPROVEMENT - Patterns significantly different")
    
    workspace = trajectory['workspace_preservation']
    print(f"\n   Workspace Preservation: {workspace['total_violation']:.3f} violation")
    if workspace['total_violation'] < 0.1:
        print("     EXCELLENT - Stays within workspace")
    elif workspace['total_violation'] < 0.5:
        print("     GOOD - Minor workspace violations")
    else:
        print("     POOR - Significant workspace violations")
    
    # Physical Deviation
    deviation = results['physical_deviation']
    print(f"\n Physical Deviation: {deviation['average']:.3f} radians (max: {deviation['maximum']:.3f})")
    if deviation['average'] < 0.2:
        print("  MINIMAL - Robot stays very close to original path")
    elif deviation['average'] < 0.5:
        print("  ACCEPTABLE - Moderate deviation from original path")
    else:
        print("  HIGH - Significant deviation from original path")
    
    # Style Characteristics
    style = results['style_characteristics']
    print(f"\n Style Characteristics Applied:")
    print(f"   Speed change: {style['speed_change']:.3f} - {'More dynamic' if style['speed_change'] > 0.2 else 'Similar tempo'}")
    print(f"   Acceleration change: {style['acceleration_change']:.3f} - {'Sharp rhythmic patterns' if style['acceleration_change'] > 0.5 else 'Gentle acceleration changes'}")
    print(f"   Coordination change: {style['coordination_change']:.3f} - {'Different joint coordination' if style['coordination_change'] > 0.2 else 'Similar coordination'}")
    print(f"   Movement range change: {style['range_change']:.3f} - {'Different movement extent' if style['range_change'] > 0.2 else 'Similar range'}")
    print(f"   Overall style transfer: {style['overall_style_change']:.3f}")
    
    if style['overall_style_change'] > 0.3:
        print("  STRONG - Clear style characteristics applied")
    elif style['overall_style_change'] > 0.1:
        print("  MODERATE - Noticeable style characteristics applied")
    else:
        print("  MINIMAL - Very little style transfer detected")
    
    # Motion Quality
    quality = results['motion_quality']
    print(f"\n Motion Smoothness: {quality['smoothness_ratio']:.3f}")
    if quality['smoothness_ratio'] < 1.2:
        print("  EXCELLENT - Motion remains smooth")
    elif quality['smoothness_ratio'] < 2.0:
        print("  FAIR - Noticeable decrease in smoothness")
    else:
        print("  POOR - Significant decrease in smoothness")
    
    # Frequency Analysis
    freq = results['frequency_analysis']
    print(f"\n Frequency Content Change: {freq['frequency_change']:.3f}")
    if freq['frequency_change'] > 0.3:
        print("  SIGNIFICANT - Notable rhythmic changes")
    elif freq['frequency_change'] > 0.1:
        print("  MODERATE - Some rhythmic changes")
    else:
        print("  MINIMAL - Little rhythmic change")
    
    # Per-Joint Analysis
    print(f"\n Per-Joint Analysis:")
    mae_vals = trajectory['absolute_accuracy']['joint_mae']
    corr_vals = trajectory['pattern_similarity']['joint_correlations']
    deviations = results['physical_deviation']['per_joint']
    
    for i, (mae, corr, dev) in enumerate(zip(mae_vals, corr_vals, deviations)):
        status = "✓" if mae < 0.3 and corr > 0.6 else "⚠" if mae < 0.5 else "✗"
        print(f"   Joint {i+1}: MAE={mae:.3f}, correlation={corr:.3f}, deviation={dev:.3f} {status}")
    
    print("="*80)

def run_enhanced_evaluation(original_csv_path, styled_csv_path, style_name=None):
    results = evaluate_style_transfer_with_reference(original_csv_path, styled_csv_path, style_name)
    
    if results:
        # Load robot data for visualisation
        original_robot = np.loadtxt(original_csv_path, delimiter=',', skiprows=1)
        styled_robot = np.loadtxt(styled_csv_path, delimiter=',', skiprows=1)
        
        # Ensure same dimensions
        min_length = min(len(original_robot), len(styled_robot))
        min_joints = min(original_robot.shape[1], styled_robot.shape[1])
        original_robot = original_robot[:min_length, :min_joints]
        styled_robot = styled_robot[:min_length, :min_joints]
        
        print_enhanced_evaluation(results)
    
    return results

In [ ]:
results = run_enhanced_evaluation(
    "/Users/pszkw/robot_motion_data/robot_motions/robot_Senario5_KatWellyTogether_Welly.csv", 
    "temporal_styled_Flamenco_c0.25_temporal_cross_attn_robot_Senario5_KatWellyTogether_Welly.csv", 
    style_name="Flamenco")